# Birge and Louveaux’s Farmer Problem

We define the abastract model containig the specification of the model in a parametric form

In [2]:
import pyomo.environ as pyo

In [15]:
from pyomo.environ import *
#-------------------------abastract model----------------------------
model = pyo.AbstractModel()

#---------------------------index sets-------------------------------
model.crops = Set()

#---------------------------parameters-------------------------------
#objective function
model.plantingCosts = Param(model.crops, within = PositiveReals)
model.purchasePrices = Param(model.crops, within = PositiveReals)
model.sellingPricesSubQuota = Param(model.crops, within = PositiveReals)

def sellingQuotaPricesValidation(model, value, i):
    return model.sellingPricesSubQuota[i] >= model.sellingPricesOverQuota[i]

model.sellingPricesOverQuota = Param(model.crops, validate = sellingQuotaPricesValidation)

#constraints
model.totalArea = Param(within = PositiveReals)
model.productionRequirement = Param(model.crops, within = NonNegativeReals)
model.pricesQuota = Param(model.crops, within = PositiveReals)
model.cropsYielding = Param(model.crops, within = NonNegativeReals)

#---------------------------variables--------------------------------
model.acresToCrops = Var(model.crops, bounds = (0.0, model.totalArea))
model.cropsPurchased = Var(model.crops, bounds = (0.0, None))
model.cropsSoldedSubQuota = Var(model.crops, bounds = (0.0, None))
model.cropsSoldedOverQuota = Var(model.crops, bounds = (0.0, None))

#--------------------------constraints-------------------------------
def totalAreaConstraint(model):
    return summation(model.acresToCrops) <= model.totalArea

def productionRequirementConstraint(model, i):
    return (model.cropsYielding[i]*model.acresToCrops[i]) + model.cropsPurchased[i]\
         - model.cropsSoldedSubQuota[i] - model.cropsSoldedOverQuota[i] >= model.productionRequirement[i]

def quotaConstraint(model, i):
    return model.cropsSoldedSubQuota[i] <= model.pricesQuota[i]

model.totalAreaConstraint = Constraint(rule = totalAreaConstraint)
model.productionRequirementConstraint = Constraint(model.crops, rule = productionRequirementConstraint)
model.quotaConstraint = Constraint(model.crops, rule = quotaConstraint)

#-----------------------objective function---------------------------
#first stage
def firstStageCost(model):
    return summation(model.plantingCosts, model.acresToCrops)

model.firstStageCost = Expression(rule = firstStageCost)

#second stage
def secondStageCost(model):
    e = summation(model.purchasePrices, model.cropsPurchased)
    e -= summation(model.sellingPricesSubQuota, model.cropsSoldedSubQuota)
    e -= summation(model.sellingPricesOverQuota, model.cropsSoldedOverQuota)
    return e

model.secondStageCost = Expression(rule = secondStageCost)

#total
def totalCost(model):
    return model.firstStageCost + model.secondStageCost

model.totalCost = Objective(rule = totalCost, sense = minimize)

We defined data necessary to initialize the abastract model

In [16]:
data = {None: {
    'crops': {None: ['wheat', 'corn', 'sugar beets']}, 
    'plantingCosts': {'wheat': 150, 'corn': 230, 'sugar beets': 260}, 
    'purchasePrices': {'wheat': 238, 'corn': 210, 'sugar beets': 100000},
    'sellingPricesSubQuota': {'wheat': 170, 'corn': 150, 'sugar beets': 36},
    'sellingPricesOverQuota': {'wheat': 170, 'corn': 150, 'sugar beets': 10},
    'totalArea': {None: 500},
    'productionRequirement': {'wheat': 200, 'corn': 240, 'sugar beets': 0},
    'pricesQuota': {'wheat': 100000, 'corn': 100000, 'sugar beets': 6000},
    'cropsYielding': {'wheat': 2.5, 'corn': 3, 'sugar beets': 20}
}
}

In [21]:
istance = model.create_instance(data=data)
istance.pprint()

1 Set Declarations
    crops : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'wheat', 'corn', 'sugar beets'}

8 Param Declarations
    cropsYielding : Size=3, Index=crops, Domain=NonNegativeReals, Default=None, Mutable=False
        Key         : Value
               corn :     3
        sugar beets :    20
              wheat :   2.5
    plantingCosts : Size=3, Index=crops, Domain=PositiveReals, Default=None, Mutable=False
        Key         : Value
               corn :   230
        sugar beets :   260
              wheat :   150
    pricesQuota : Size=3, Index=crops, Domain=PositiveReals, Default=None, Mutable=False
        Key         : Value
               corn : 100000
        sugar beets :   6000
              wheat : 100000
    productionRequirement : Size=3, Index=crops, Domain=NonNegativeReals, Default=None, Mutable=False
        Key         : Value
               corn :   240
        sugar beets

We solve the model using the glpk solver

In [39]:
import os

os.environ['NEOS_EMAIL'] = 'marco.scatassi.99@gmail.com' 

opt = SolverManagerFactory('neos')opt.solve(istance, solver = 'cplex')

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'CPLEX 20.1.0.0\\x3a optimal solution; objective -118600; 7 dual simplex iterations (5 in phase I)', 'Termination condition': 'optimal', 'Id': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [54]:
istance.display()

Model unknown

  Variables:
    acresToCrops : Size=3, Index=crops
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
               corn :   0.0 :  80.0 :   500 : False : False :  Reals
        sugar beets :   0.0 : 300.0 :   500 : False : False :  Reals
              wheat :   0.0 : 120.0 :   500 : False : False :  Reals
    cropsPurchased : Size=3, Index=crops
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
               corn :   0.0 :   0.0 :  None : False : False :  Reals
        sugar beets :   0.0 :   0.0 :  None : False : False :  Reals
              wheat :   0.0 :   0.0 :  None : False : False :  Reals
    cropsSoldedSubQuota : Size=3, Index=crops
        Key         : Lower : Value  : Upper : Fixed : Stale : Domain
               corn :   0.0 :    0.0 :  None : False : False :  Reals
        sugar beets :   0.0 : 6000.0 :  None : False : False :  Reals
              wheat :   0.0 :    0.0 :  None : False : False :  Reals
    cropsSolde